In [1]:
# 라이브러리 호출

import matplotlib.pyplot as plt
import os
import shutil
import glob
import cv2

import tensorflow as tf
import numpy as np
import pandas as pd

import time

In [3]:
# 경로 지정하기

# 파일이 있는 경로
train_dir = '../dogs-vs-cats/train/train'

# train용 폴더 경로
train_set_dir = os.path.join(train_dir, 'train_set')
train_dog_dir = os.path.join(train_set_dir, 'dog')
train_cat_dir = os.path.join(train_set_dir, 'cat')

# valid용 폴더 경로
valid_set_dir = os.path.join(train_dir, 'valid_set')
valid_dog_dir = os.path.join(valid_set_dir, 'dog')
valid_cat_dir = os.path.join(valid_set_dir, 'cat')

# test용 폴더 경로
test_set_dir = os.path.join(train_dir, 'test_set')
test_dog_dir = os.path.join(test_set_dir, 'dog')
test_cat_dir = os.path.join(test_set_dir, 'cat')

# training set 20000장(cat:10000/dog:10000), cross-validation set 4000장(cat:2000/dog:2000), test set 1000장(cat:500/dog:500)
print(f'the number of train set : {len(os.listdir(train_dog_dir)) + len(os.listdir(train_cat_dir))}')
print(f'the number of validn set : {len(os.listdir(valid_dog_dir)) + len(os.listdir(valid_cat_dir))}')
print(f'the number of test set : {len(os.listdir(test_dog_dir)) + len(os.listdir(test_cat_dir))}')

the number of train set : 20000
the number of validn set : 4000
the number of test set : 1000


In [4]:
# ImageDataGenerator를 통해서 data generator를 생성

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, # nomalization 적용
                                                                rotation_range=40,# 사진 회전 각도 범위 (0~180)
                                                                width_shift_range=0.2, #수평 이동 범위(비율값)
                                                                height_shift_range=0.2, #수직 이동 범위(비율값)
                                                                shear_range=0.2,# 전단변환을 적용할 각도 범위; 사진을 3d로 기울임
                                                                zoom_range=0.2, # 사진 확대 범위
                                                                horizontal_flip=True, # 랜덤하게 이미지를 수평으로 뒤집음
                                                                fill_mode='nearest') # 회전이나 이동을 통해 빈 곳이 생기면 픽셀을 채움(nearest는 인접합 픽셀 사용)

# rescale 파라미터를 이용해 모든 데이터 255로 나누어주기
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# flow_from_directory() 메서드를 이용해서 훈령과 테스트에 사용될 이미지 데이터 만들기
train_generator = train_datagen.flow_from_directory(train_set_dir, # 이미지들이 위치한 경로
                                                    target_size=(150,150), # 이미지의 크기 조절
                                                    batch_size=64,
                                                    class_mode='binary')

valid_generator = valid_datagen.flow_from_directory(valid_set_dir,
                                                    target_size=(150,150),
                                                    batch_size=64,
                                                    class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_set_dir,
                                                  target_size=(150,150),
                                                  batch_size=64,
                                                  class_mode='binary')

train_step = train_generator.n // 64
valid_step = valid_generator.n // 64
test_step = test_generator.n // 64


Found 20000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [5]:
print('train_step:', train_step)
print('valid_step:',valid_step)
print('test_step:', test_step)

train_step: 312
valid_step: 62
test_step: 15


In [6]:
# cnn 모델 구현

model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=(150,150,3)),
  tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=1, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=1, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=1, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=1, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(1024, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(1, activation='sigmoid')
])


model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 128)       1

In [7]:
# 최적화

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), # 정규화? 최적화? 훈련과정 설정. 최적화 알고리즘 설정
              loss='binary_crossentropy', # 모델 최적화에 사용되는 목적함수
              metrics=['acc']) # 훈련을 모니터링 하기위해 사용

# 모델 학습하기
''' fit() 과 fit_generator() 차이?
fit() : 사이킷런의 fit 메소드와 유사. 학습에 사용할 데이터 x와 y 전체를 한번에 입력으로 사용 -> 메모리 많이 사용
fit_generator() : 파이썬의 generator를 사용한 것. 대용량을 데이터를 효율적으로 학습하기 위한 것
파이썬의 generator를 통해 형성된 데이터들을 batch-by-batch로 학습하는 방법 -> cpu를 parallel(평행? 병렬?)하게 사용할 때 효율적
'''

start= time.time()
model.fit_generator(train_generator,
                    steps_per_epoch=train_step, # steps_per_epoch : 한 번의 epoch에서 훈련에 사용할 batch의 개수 지정
                    epochs=30, # epoch : 데이터셋을 한 번 훈련하는 과정
                    validation_data=valid_generator,
                    validation_steps=valid_step # validation_steps : 한 번의 epoch이 끝날 때, 테스트에 사용되는 batch의 개수 지정
                    )


# model.fit(train_generator,
#                     steps_per_epoch=train_step,
#                     epochs=30,
#                     validation_data=valid_generator,
#                     validation_steps=valid_step
#                     )

print('==========================================================================================================================================')
print('time :', time.time()-start)
print('==========================================================================================================================================')

# 테스트용 데이터로 평가

model.save('CNN_epoch_30_batch_64.h5')

test_loss, test_acc = model.evaluate_generator(test_generator,
                                               steps=test_step,
                                               workers=4)

print(f'test loss : {test_loss:.4f} / test acc : {test_acc*100:.2f} %')


C:\Users\Public\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
312/312 [==============================] - 495s 2s/step - loss: 0.6751 - acc: 0.5620 - val_loss: 0.6563 - val_acc: 0.6190
Epoch 2/30
312/312 [==============================] - 458s 1s/step - loss: 0.6562 - acc: 0.6097 - val_loss: 0.6829 - val_acc: 0.6114
Epoch 3/30
312/312 [==============================] - 456s 1s/step - loss: 0.6027 - acc: 0.6738 - val_loss: 0.5028 - val_acc: 0.7513
Epoch 4/30
312/312 [==============================] - 467s 1s/step - loss: 0.5541 - acc: 0.7161 - val_loss: 0.5205 - val_acc: 0.7457
Epoch 5/30
312/312 [==============================] - 461s 1s/step - loss: 0.5358 - acc: 0.7284 - val_loss: 0.4500 - val_acc: 0.7961
Epoch 6/30
312/312 [==============================] - 464s 1s/step - loss: 0.5150 - acc: 0.7431 - val_loss: 0.4595 - val_acc: 0.7840
Epoch 7/30
312/312 [==============================] - 457s 1s/step - loss: 0.5014 - acc: 0.7552 - val_loss: 0.4243 - val_acc: 0.8110
Epoch 8/30
312/312 [==============================] - 458s 1s/step - 

C:\Users\Public\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


test loss : 0.2127 / test acc : 90.00 %


In [8]:
13832.005680561066/3600

3.8422238001558515

In [9]:
# 최적화

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), # 정규화? 최적화? 훈련과정 설정. 최적화 알고리즘 설정
              loss='binary_crossentropy', # 모델 최적화에 사용되는 목적함수
              metrics=['acc']) # 훈련을 모니터링 하기위해 사용

# 모델 학습하기
''' fit() 과 fit_generator() 차이?
fit() : 사이킷런의 fit 메소드와 유사. 학습에 사용할 데이터 x와 y 전체를 한번에 입력으로 사용 -> 메모리 많이 사용
fit_generator() : 파이썬의 generator를 사용한 것. 대용량을 데이터를 효율적으로 학습하기 위한 것
파이썬의 generator를 통해 형성된 데이터들을 batch-by-batch로 학습하는 방법 -> cpu를 parallel(평행? 병렬?)하게 사용할 때 효율적
'''

start= time.time()
model.fit_generator(train_generator,
                    steps_per_epoch=train_step, # steps_per_epoch : 한 번의 epoch에서 훈련에 사용할 batch의 개수 지정
                    epochs=50, # epoch : 데이터셋을 한 번 훈련하는 과정
                    validation_data=valid_generator,
                    validation_steps=valid_step # validation_steps : 한 번의 epoch이 끝날 때, 테스트에 사용되는 batch의 개수 지정
                    )


# model.fit(train_generator,
#                     steps_per_epoch=train_step,
#                     epochs=30,
#                     validation_data=valid_generator,
#                     validation_steps=valid_step
#                     )

print('==========================================================================================================================================')
print('time :', time.time()-start)
print('==========================================================================================================================================')

# 테스트용 데이터로 평가

model.save('CNN_epoch_30_batch_64.h5')

test_loss, test_acc = model.evaluate_generator(test_generator,
                                               steps=test_step,
                                               workers=4)

print(f'test loss : {test_loss:.4f} / test acc : {test_acc*100:.2f} %')


Epoch 1/50
312/312 [==============================] - 624s 2s/step - loss: 0.2467 - acc: 0.8963 - val_loss: 0.2130 - val_acc: 0.9115
Epoch 2/50
312/312 [==============================] - 486s 2s/step - loss: 0.2408 - acc: 0.8986 - val_loss: 0.1768 - val_acc: 0.9312
Epoch 3/50
312/312 [==============================] - 477s 2s/step - loss: 0.2362 - acc: 0.8996 - val_loss: 0.2176 - val_acc: 0.9115
Epoch 4/50
312/312 [==============================] - 461s 1s/step - loss: 0.2404 - acc: 0.8993 - val_loss: 0.2065 - val_acc: 0.9178
Epoch 5/50
312/312 [==============================] - 456s 1s/step - loss: 0.2322 - acc: 0.9018 - val_loss: 0.1989 - val_acc: 0.9115
Epoch 6/50
312/312 [==============================] - 477s 2s/step - loss: 0.2277 - acc: 0.9043 - val_loss: 0.1873 - val_acc: 0.9259
Epoch 7/50
312/312 [==============================] - 487s 2s/step - loss: 0.2241 - acc: 0.9060 - val_loss: 0.1941 - val_acc: 0.9236
Epoch 8/50
312/312 [==============================] - 483s 2s/step - 